## **Projet Walmart**

Importation des librairies nécessaires pour le projet 

In [52]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

import datetime as dt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder 
from sklearn.compose import ColumnTransformer
from sklearn import metrics


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning )
warnings.filterwarnings("ignore", category=FutureWarning )
warnings.filterwarnings("ignore", category=UserWarning )
warnings.filterwarnings("ignore", message="Pandas requires version '1.3.6' or newer of 'bottleneck'")
warnings.simplefilter(action='ignore', category=FutureWarning)


## Partie 1 : réaliser une EDA et tous les prétraitements nécessaires pour préparer les données pour le machine learning

## A - Importation du dataset et statistique basiques 

In [53]:
data= pd.read_csv("Walmart_Store_sales.csv")
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [54]:
# Statistique basique
print("Number of rows : {}".format(data.shape[0]))
print()

print("Display of dataset: ")
display(data.head())
print()

print("Basics statistics: ")
data_desc = data.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*data.isnull().sum()/data.shape[0])

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

## B- data cleanning

In [55]:
# mettre la date dans le bon format 
data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y', errors='coerce')

In [56]:
# Remove NaN in Weekly_Sales
df = data.copy(deep=True)
df = df.dropna(subset=['Weekly_Sales'])

display(100*df.isnull().sum()/df.shape[0])
display(df.shape)

Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
dtype: float64

(136, 8)

In [57]:
# Create datetime type columns
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')


df['day_of_week'] = df['Date'].dt.weekday
df['day_of_month'] = df['Date'].dt.day
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month

df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,day_of_week,day_of_month,year,month
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,4.0,18.0,2011.0,2.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,4.0,25.0,2011.0,3.0
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,4.0,28.0,2010.0,5.0
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,4.0,28.0,2010.0,5.0


In [58]:
# supprimer les colonnes date et dayofweek
useless_column = ['Date', 'day_of_week']
df = df.drop(useless_column, axis=1)

In [59]:
# Vérifier si la colonne 'dayofweek' existe dans le DataFrame
if 'day_of_week' in data.columns:
    # Supprimer la colonne 'dayofweek' si elle existe
    data = data.drop('dayofweek', axis=1)
    print("La colonne 'dayofweek' a été supprimée avec succès.")
else:
    print("La colonne 'dayofweek' n'existe pas dans le DataFrame.")

La colonne 'dayofweek' n'existe pas dans le DataFrame.


In [60]:
# Remove outliers
remove_outlier = ['Temperature','Fuel_Price','CPI','Unemployment']

for col in remove_outlier:
    mean = df[col].mean()
    std = df[col].std()

    mask = np.abs((df[col] - mean) <= 3* std) | ((df[col] - mean) >= - 3* std)
    df = df[mask]

In [61]:
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,day_of_month,year,month
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,18.0,2011.0,2.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,25.0,2011.0,3.0
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,28.0,2010.0,5.0
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,3.0,2011.0,6.0
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,3.0,2012.0,2.0


## PARTIE 2 : EDA 

In [62]:
# Regrouper les  magasin et calculer la moyenne des ventes hebdomadaires
weekly_sales_mean = df.groupby('Store').agg({'Weekly_Sales':'mean'}).reset_index()

fig = px.bar(weekly_sales_mean, 
             x='Store', 
             y='Weekly_Sales', 
             title="Ventes hebdomadaires moyennes par magasin", 
             color='Weekly_Sales',
             color_continuous_scale=px.colors.sequential.Viridis)  


fig.update_xaxes(title="Store")
fig.update_yaxes(title="Average weekly sales")


fig.show()

In [63]:
# Create a bar graph for the average sales when there is a holiday or not in the week
weekly_sales_mean = df.groupby(['Holiday_Flag']).agg({'Weekly_Sales':'mean'})
fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index,
              y='Weekly_Sales',
                title="Average weekly sales when there is a holiday or not", 
                color='Weekly_Sales',color_continuous_scale=px.colors.sequential.Viridis)
fig.update_xaxes(title="Holiday_flag")
fig.update_yaxes(title="Average weekly sales")
fig.show()

In [64]:
# Create a bar graph of the average weekly sale per month
weekly_sales_mean = df.groupby(['month']).agg({'Weekly_Sales':'mean'})
fig = px.bar(weekly_sales_mean, x=weekly_sales_mean.index, y='Weekly_Sales', title="Average weekly sales per month", color='Weekly_Sales',color_continuous_scale=px.colors.sequential.Viridis)
fig.update_xaxes(title="month")
fig.update_yaxes(title="Average weekly sales")
fig.show()

In [65]:
# Create a bar graph of the average weekly sales per day in a month
weekly_sales_mean = df.groupby(['day_of_month']).agg({'Weekly_Sales':'mean'})

fig = px.bar(weekly_sales_mean,
x=weekly_sales_mean.index,
              y='Weekly_Sales', title="Average weekly sales per day", 
              color='Weekly_Sales',color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title="Day")
fig.update_yaxes(title="Average weekly sales")
fig.show()

In [66]:
weekly_sales_mean = df.groupby('year').agg({'Weekly_Sales':'mean'}).reset_index()

fig = px.pie(weekly_sales_mean, 
             names='year', 
             values='Weekly_Sales', 
             title="Average weekly sales per year",
             color='year',
             color_discrete_sequence=['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A'])  

fig.show()

In [67]:
# Create a bar graph of the average weekly sale per year
weekly_sales_mean = df.groupby(['year']).agg({'Weekly_Sales':'mean'})

fig = px.bar(weekly_sales_mean,x=weekly_sales_mean.index, 
              y='Weekly_Sales', 
              title="Average weekly sales per year",
                color='Weekly_Sales',color_continuous_scale=px.colors.sequential.Viridis)

fig.update_xaxes(title="year")
fig.update_yaxes(title="Average weekly sales")
fig.show()

In [68]:
# Create a scatter graph: impact of temperature on weekly sales
fig = px.scatter(df, x='Temperature', y='Weekly_Sales', trendline="ols", title="Weekly sales based on temperature")
fig.show()

In [69]:
# Create a scatter graph: impact of fuel prices on weekly sales
fig = px.scatter(df, x='Fuel_Price', y='Weekly_Sales', trendline="ols", title="Weekly sales based on fuel price")
fig.show()

In [70]:
# Create a correlation matrix
import plotly.figure_factory as ff
corr_matrix = df.corr().round(2)
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x=corr_matrix.columns.tolist(),
                                  y=corr_matrix.index.tolist())

fig.update_layout(title_text='Correlation matrix')

fig.show()

Partie 2 : Entraîner un modèle de régression linéaire 

In [71]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'

print("Separating labels from features...")
Y = df.loc[:,target_name]
X = df.drop(target_name, axis=1)
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
6   15.0           0.0        69.80       4.069  134.855161         7.658   
7   20.0           0.0        39.93       3.617  213.023622         6.961   

   day_of_month    year  month  
0          18.0  2011.0    2.0  
1          25.0  2011.0    3.0  
4          28.0  2010.0    5.0  
6           3.0  2011.0    6.0  
7           3.0  2012.0    2.0  



In [72]:
# Train, test split
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [73]:
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'day_of_month', 'year', 'month']
numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
])

In [74]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5])
print()

Performing preprocessings on train set...
    Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
64    4.0           0.0        36.44       3.149  129.898065         5.143   
39    5.0           0.0        60.71       3.297  218.569962         6.300   
62   12.0           0.0        52.77       3.236  126.879484        14.313   
36    3.0           0.0        80.19       3.467  219.741491         7.567   
10    8.0           0.0        82.92       3.554  219.070197         6.425   

    day_of_month    year  month  
64          16.0  2011.0   12.0  
39          11.0  2011.0   11.0  
62          17.0  2010.0   12.0  
36          23.0  2011.0    9.0  
10          19.0  2011.0    8.0  
...Done.
[[-1.4929771  -0.41143513 -1.04170369 -1.40925613  0.04184723  0.14063251
   1.92858269  0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0

In [75]:
# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test)

print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on test set...
    Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
4     6.0           0.0        78.89       2.759  212.412888         7.092   
45    2.0           0.0        54.63       3.555  220.275944         7.057   
82   11.0           0.0        52.77       3.510  223.917015         6.833   
23   19.0           0.0        39.07       2.963  131.863129         8.350   
20    7.0           0.0        38.26       2.725  189.704822         8.963   

    day_of_month    year  month  
4           28.0  2010.0    5.0  
45          24.0  2012.0    2.0  
82           NaN     NaN    NaN  
23          26.0  2010.0    3.0  
20           2.0  2010.0    4.0  
...Done.
[[ 0.95346505 -1.22821791  0.9920462  -0.31547104  1.54834758 -1.17779728
  -0.49368026  0.          0.          0.          0.          1.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.

In [76]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

# Make predictions
Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)

Train model...
...Done.


In [77]:
# Print R2 score
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9816450826863105
R2 score on test set :  0.9473578763934809


In [78]:
# Cross-validation
scores = cross_val_score(regressor, X_train, Y_train, cv=10)
avg = scores.mean()
std = scores.std()
print("Cross-validation accuracy:", avg)
print("Cross-validation standard deviation:", std)

Cross-validation accuracy: 0.9445320913142135
Cross-validation standard deviation: 0.02814016528446571


In [79]:
# Features importance
print(regressor.coef_)

[  -49564.89180929   -75905.4042442    662027.67253298   -84369.02896666
   -51149.79119519   -21814.95527375    34387.04018479   170518.40921647
 -1303307.67665164  1945496.41343119 -1342424.44665849    22140.7085293
  -619826.84426965  -746768.58966428 -1259443.39966329  1708326.89755824
  1212194.55663425  1778265.09692492  1003916.78215621   445935.4823868
  -756512.97533723   476135.70543875   860735.78164533  1140004.5702776
   519787.83183498  -167301.97731309]


In [80]:
# Name the features
column_names = []
for name, pipeline, features_list in preprocessor.transformers_:
    if name == 'num': 
        features = features_list 
    else: 
        features = pipeline.named_steps['encoder'].get_feature_names_out() 
    column_names.extend(features) 
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'day_of_month', 'year', 'month', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_12.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


In [81]:
coefs = pd.DataFrame(index=column_names, data=regressor.coef_.transpose(), columns=["coefficients"])
fig = px.bar(coefs, x = coefs.index, y = "coefficients")
fig.show()

Partie 3 : éviter le surajustement en entraînant un modèle de régression régularisé

In [82]:
# Performing grid search on Lasso
print("Grid search...")
regressor = Lasso(max_iter=100000)

params = {
    'alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 1.5, 2.5, 5.0, 10.0, 20.0]
}
linreg_cv = GridSearchCV(regressor, param_grid=params, cv=10)
linreg_cv.fit(X_train, Y_train)
print("...Done.")
print("")
print("Tuned Linear Regression Parameters: {}".format(linreg_cv.best_params_))
print("Best score is {}".format(linreg_cv.best_score_))
print("")
print("R2 score on training set : ", linreg_cv.score(X_train, Y_train))
print("R2 score on test set : ", linreg_cv.score(X_test, Y_test))

Grid search...
...Done.

Tuned Linear Regression Parameters: {'alpha': 10.0}
Best score is 0.945324754989087

R2 score on training set :  0.9815591107287789
R2 score on test set :  0.9479540933952665


In [83]:
# Performing grid search on Ridge
print("Grid search...")
regressor = Ridge()

params = {
    'alpha': [0.0, 0.1, 0.5, 1.0]
}
linreg_cv = GridSearchCV(regressor, param_grid=params, cv=10)
linreg_cv.fit(X_train, Y_train)
print("...Done.")
print("")
print("Tuned Linear Regression Parameters: {}".format(linreg_cv.best_params_))
print("Best score is {}".format(linreg_cv.best_score_))
print("")
print("R2 score on training set : ", linreg_cv.score(X_train, Y_train))
print("R2 score on test set : ", linreg_cv.score(X_test, Y_test))

Grid search...
...Done.

Tuned Linear Regression Parameters: {'alpha': 0.0}
Best score is 0.9445320913142158

R2 score on training set :  0.9816450826863105
R2 score on test set :  0.9473578763934762


#### En conclusion, notre analyse comparative entre la régression linéaire simple, Ridge et Lasso révèle des performances similaires, avec une légère préférence pour le modèle de Lasso qui semble se généraliser légèrement mieux sur l’ensemble de test. Cependant, la différence entre les modèles est minime, ce qui souligne la robustesse globale de l’approche linéaire pour la prédiction dans notre contexte spécifique.